In [46]:
#!git clone https://github.com/Panizzutti/GPT_from_scratch.git

In [47]:
import torch

In [48]:
with open('divina_commedia.txt', 'r', encoding = 'utf-8') as f:
    text = f.read()


In [49]:
print(text[:200])

LA DIVINA COMMEDIA
di Dante Alighieri
INFERNO



Inferno: Canto I

  Nel mezzo del cammin di nostra vita
mi ritrovai per una selva oscura
ché la diritta via era smarrita.
  Ahi quanto a dir qual era è


In [50]:
len(text)

537093

In [51]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("".join(chars))
print(vocab_size)



 !"'(),-.:;?ABCDEFGHILMNOPQRSTUVXZabcdefghijlmnopqrstuvxyz~àèéìïòóù
68


In [52]:
char_to_int = { ch: i for i,ch in enumerate(chars)}
int_to_char = { i: ch for i,ch in enumerate(chars)}
encode = lambda s: [char_to_int[c] for c in s] #encoder function
decode = lambda l: "".join(int_to_char[i] for i in l) #decoder function

#not byte pair encoding, for simplicity

In [53]:
decode(encode("cianoxxj"))


'cianoxxj'

In [54]:
encode("ciao mondo")

[37, 43, 35, 48, 1, 46, 48, 47, 38, 48]

In [55]:
data = torch.tensor(encode(text))

In [56]:
n = int(0.9 * len(data))
train_data= data[:n]
val_data= data[n:]


In [57]:
batch_size = 4
block_size= 10

train_data[:block_size+1]

tensor([22, 13,  1, 16, 21, 32, 21, 24, 13,  1, 15])

In [58]:
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 10])
tensor([[43,  1, 55, 48, 52, 53, 51, 43,  1, 52],
        [ 9,  0,  1,  1, 17,  1, 43, 48,  7,  1],
        [ 1, 49, 51, 39, 37, 48, 11,  0, 39,  1],
        [48, 43,  0, 35, 45, 45, 48, 51, 35,  1]])
targets:
torch.Size([4, 10])
tensor([[ 1, 55, 48, 52, 53, 51, 43,  1, 52, 37],
        [ 0,  1,  1, 17,  1, 43, 48,  7,  1, 37],
        [49, 51, 39, 37, 48, 11,  0, 39,  1, 52],
        [43,  0, 35, 45, 45, 48, 51, 35,  1, 37]])
----
when input is [43] the target: 1
when input is [43, 1] the target: 55
when input is [43, 1, 55] the target: 48
when input is [43, 1, 55, 48] the target: 52
when input is [43, 1, 55, 48, 52] the target: 53
when input is [43, 1, 55, 48, 52, 53] the target: 51
when input is [43, 1, 55, 48, 52, 53, 51] the target: 43
when input is [43, 1, 55, 48, 52, 53, 51, 43] the target: 1
when input is [43, 1, 55, 48, 52, 53, 51, 43, 1] the target: 52
when input is [43, 1, 55, 48, 52, 53, 51, 43, 1, 52] the target: 37
when input is [9] the target:

In [59]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(42)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
m

BigramLanguageModel(
  (token_embedding_table): Embedding(68, 68)
)

In [60]:
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([40, 68])
tensor(4.3539, grad_fn=<NllLossBackward0>)


N)QNAlBPdìl~NnézfGCf.ehAEèòz(BèOdBz(UòlN~McXtRFjZHD!ìxZbìRbqóïOdMè,x!òx!eZbU)~b yXTQ?GgP?-DnCDùSùz~
